# 1   Introduction
### Welcome to SparkBeyond's Airbnb Demo!

<img src="f_airbnb.png" style="width: 100px;">

In this demo, we will predict the price of a Airbnb's listing using various SparkBeyond functionalities. You will see Sparkbeyond's capabilities of finding drivers and insights underlying the rental price. <font color=red> Add more contents here (eg. the functionality we covered, basic, geo, text, image)</font>

The demo is divided in to 9 main sections as shown below.

### Table of Contents
> 1 Introduction

> 2 Data Exploration  
    >> 2.1 Import the Data  
    >> 2.2 Visualize the Data
    
> 3 Price Prediction with Primary Dataset
    >> 3.1 Simple Baseline Model  
    >> 3.2 Advanced - Add Column Interactions

> 4 Census Dataset  

> 5 Open Street Map  

> 6 Text Analysis  

> 7 Azure Image Analysis  

> 8 Bring everything together

> 9 Summary

# 2   Data Exploration
## 2.1 Import the data

---
Our training data is collected from the Inside Airbnb website (http://insideairbnb.com/), which is an independent, non-commercial set of tools and data that allows people to explore how Airbnb is being used in cities around the world. <font color=red>Adam: Do you think it is necessary to add some information about your data prepartion process  (e.g. what states are chosen, any special consideration when generating the whole datasets, etc.)? </font>

Despite the wide range of data provided, in this demo, we will focus on the Airbnb rental in New York state. Airbnb provides three types of rooms: entire home/apt, private room and shared room. Here, we will specifically predict the rental price for the entire home/apt.

---

In [ ]:
import pandas as pd 
import sparkbeyond as sb 
import matplotlib.pylab as plt
pd.set_option('display.max_columns',None)

In [ ]:
#SB

In [ ]:
# Make a sparkbeyond client instance 
server_url    = 'https://training2.sparkbeyond.com/'
username     = 'ryan.grosso@sparkbeyond.com'
password = 'Raiders72!'

client = sb.SparkBeyondClient(server_url = server_url,
                              username = username,
                              password = password)
proj_name = 'Airbnb_nyc_santander_demo'

## Load the data
explore the data to understand what is availible 

In [ ]:
#Bring in data here
df_all = pd.read_csv('../../data/airbnb_nyc_santander.csv')

In [ ]:
print(df_all.shape)
print(df_all.columns)

In [ ]:
df_all.head(1)

In [ ]:
# Take a look at the target 

In [ ]:
df_all.PRICE.hist(bins=20,log=True)

---
In the our primary dataset, we have selected a subset of columns from the raw data and filter out listings with price outside the range of \$50 to \$500. The columns with a lot of texts are excluded for now and will be further investigated in Section 6. We keep the listing URLs so that we can easily find the actual listing on Airbnb's website when investigating specific samples, but the URLs will be removed later before running a pipeline.

Below is the data dictionary for this dataset.

| Column Name   |      Content      |
| ---- | ---- |
| listing_url |  The URL for the listing |
| neighbourhood_cleansed |  When the data is generated by Inside Airbnb, neighbourhood names for each listing are compiled by comparing the listing's geographic coordinates with a city's definition of neighbourhoods. Airbnb neighbourhood names are not used because of their inaccuracies |
| neighbourhood_group_cleansed | Similar to 'neighbourhood_cleansed', this column includes cleaned data that captures the district at which the listing is located | 
| city | City at which the listing is located | 
| zipcode | Zipcode of the listing | 
| latitude | Latitude of the listing | 
| longitude | Longitude of the listing | 
| property_type | Type of the property, e.g. apartment, house, loft etc.| 
| accommodates | Number of individuals that can live concurrently in the listing | 
| bathrooms | Number of bathrooms in the listing | 
| bedrooms | Number of bedrooms in the listing | 
| beds | Number of beds in the listing | 
| bed_type | Type of the bed, e.g. real bed, pull-out sofa, futon etc. | 
| amenities | All the amenities (i.e. complement services such as Internet, TV, dishwasher) of the listing | 
| square_feet | Area of the listing in the unit of square feet | 
| price | Price of the listing | 
| minimum_nights | Number of minimum nights that the listing must be rented | 
| maximum_nights | Number of maximum nights that the listing can be rented | 
| number_of_reviews | Total number of reviews from guests that the listing has received from its publication | 
| review_scores_rating | Average rating of this listing (ranging from 0 to 100) | 
| cancellation_policy | Requirements that need to be fulfilled in order to cancel the reservation and be able to recover the payment | 
| reviews_per_month | Average number of reviews received per month | 

---

Finally let's sample our data
For SparkBeyond, we nee

In [ ]:
df = df_all.sample(n=6000,random_state=42)

## Primary Data Pipeline
First build a simple pipeline using simple data columns that are easily interpretable. 

In [ ]:
primary_columns = ['PRICE',"listing_url",'name',"zipcode",'property_type','accommodates','bathrooms', 'bedrooms', 'beds', 'bed_type','amenities']
df_primary = df[primary_columns]

In [ ]:
%%capture
#baseline for private rooms
model_all_base = client.learn(                 
     project_name = proj_name,            # project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df_primary,             # If test data is not specified, it will be randomly generated from  
     target='PRICE',                      # target to predict
     revision_description = 'base model', # Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished

Some features that are not expressed in a useful way. Sparkbeyond has the ablilty to blacklist features. This means that the features will not be searching during features search 

In [ ]:
#Functionality of features search is controled through the feature generaion node
my_feature_gen = sb.FeatureGeneration(functions_black_list=['consecutiveCharactersText','punctuationRatio','upperCaseRatio','letterUpperCaseRatio'],
                                      allow_range_features=False,
                                      numeric_equality_features=False,equality_features=False)

In [ ]:
%%capture
#baseline for private rooms
model_all_base_feat = client.learn(                 
     project_name = proj_name,            # Will be used as the project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df_primary,               # If test data is not specified, it will be randomly generated from  
     feature_generation=my_feature_gen,   # train_data according to the default settings (80% train, 20% test )
     target='PRICE',                      # target to predict
     revision_description ='Data for NYC Baseline',# Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished

## Text only


In [ ]:
primary_text_columns = ['PRICE',"listing_url",'name',"summary",'property_type','space','description', 'neighborhood_overview', 'notes', 'bed_type','transit','amenities','access','interaction','house_rules']
df_text = df[primary_text_columns]

In [ ]:
my_feature_gen = sb.FeatureGeneration(functions_black_list=['consecutiveCharactersText','punctuationRatio','upperCaseRatio','letterUpperCaseRatio'],
                                      allow_range_features=False,numeric_equality_features=False,equality_features=True)

In [ ]:
%%capture
#baseline for private rooms
model_text_base = client.learn(                 
     project_name = proj_name,            # Will be used as the project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df_text,               # If test data is not specified, it will be randomly generated from  
     feature_generation=my_feature_gen,   # train_data according to the default settings (80% train, 20% test )
     target='PRICE',                      # target to predict
     revision_description ='Data for NYC Adjusting text',# Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished

# now let's add worldknowedge using SparkBeyond's linked data core
What is world knowledge?

In [ ]:
knowledge_object = sb.Knowledge(linked_data_core=True)

In [ ]:
%%capture
#baseline for private rooms
model_text_base = client.learn(                 
     project_name = proj_name,            # Will be used as the project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df_text,               # If test data is not specified, it will be randomly generated from  
     feature_generation=my_feature_gen,   # train_data according to the default settings (80% train, 20% test )
     target='PRICE',                      # target to predict
    knowledge=knowledge_object,
     revision_description ='Data for NYC Adjusting text linked data core',# Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished

## GeoSpatial

In [ ]:
primary_geo_columns = ['PRICE',"listing_url",'latitude','longitude','zipcode']
df_geo = df[primary_geo_columns]

In [ ]:
my_feature_gen = sb.FeatureGeneration(functions_black_list=['consecutiveCharactersText','punctuationRatio','upperCaseRatio','letterUpperCaseRatio'],
                                      allow_range_features=False,numeric_equality_features=False,equality_features=False)

In [ ]:
%%capture
#geo
model_text_base = client.learn(                 
     project_name = proj_name,            # Will be used as the project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df_geo,               # If test data is not specified, it will be randomly generated from  
     feature_generation=my_feature_gen,   # train_data according to the default settings (80% train, 20% test )
     target='PRICE',                      # target to predict
     revision_description ='Data for NYC Adjusting  geo',# Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished

In [ ]:
# Add geospatial context 

In [ ]:
geospatial_context = sb.Contexts.GeoSpatialContext(
    # when adding data make sure what you are adding is reasonable. 
    # using Ints/Doubles will increase the speed.... adding text/date columns takes more time and doesn't always make sense
    data = df_geo[['PRICE','latitude','longitude']],
    name = 'geospatial_cont'
)   

In [ ]:
%%capture
#geo
model_text_base = client.learn(                 
     project_name = proj_name,            # Will be used as the project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df_geo,               # If test data is not specified, it will be randomly generated from  
     feature_generation=my_feature_gen,   # train_data according to the default settings (80% train, 20% test )
     target='PRICE',                      # target to predict
     context_datasets=  [geospatial_context], # Add Context data sets
     revision_description ='Data for NYC Adjusting geo',# Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished

### Adding census is just another context 

In [ ]:
%%capture
#geo
model_geo_census = client.learn(                 
     project_name = proj_name,            # Will be used as the project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df_geo,               # If test data is not specified, it will be randomly generated from  
     feature_generation=my_feature_gen,   # train_data according to the default settings (80% train, 20% test )
     target='PRICE',                      # target to predict
     context_datasets=  [geospatial_context,client.world_knowledge.get.Census], # Add Context data sets as a list of context datasets
     revision_description ='Data for NYC Adjusting geo',# Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished

In [ ]:
# Add Open Street map

In [ ]:
knowledge_object = sb.Knowledge(
    open_street_map  = True,
)

In [ ]:
%%capture
#geo
model_geo_osm = client.learn(                 
     project_name = proj_name,            # Will be used as the project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df_geo,               # If test data is not specified, it will be randomly generated from  
     feature_generation=my_feature_gen,   # train_data according to the default settings (80% train, 20% test )
     target='PRICE',                      # target to predict
    knowledge=knowledge_object,
     context_datasets =  [geospatial_context,client.world_knowledge.get.Census], # Add Context data sets as a list of context datasets
     revision_description ='Data for NYC Adjusting geo and osm',# Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished

## Full dataset 
Build a model using the entire dataset


In [ ]:
geospatial_context = sb.Contexts.GeoSpatialContext(
    # when adding data make sure what you are adding is reasonable. 
    # using Ints/Doubles will increase the speed.... adding text/date columns takes more time and doesn't always make sense
    data = df_geo[['PRICE','latitude','longitude']],
    name = 'geospatial_cont'
)   
# Census datasets
census_data_context = client.world_knowledge.get.Census

# all the knowledge
knowledge_object_all = sb.Knowledge(
    linked_data_core=True,
    open_street_map  = True,
)

In [ ]:
%%capture
#geo
model_geo_osm = client.learn(                 
     project_name = proj_name,            # Will be used as the project name. When executing again with the same name,
                                          # a new revision for the same project will be created
     train_data = df,               # If test data is not specified, it will be randomly generated from  
     feature_generation=my_feature_gen,   # train_data according to the default settings (80% train, 20% test )
     target='PRICE',                      # target to predict
    knowledge=knowledge_object_all,
     context_datasets =  [geospatial_context,census_data_context], # Add Context data sets as a list of context datasets
     revision_description ='Data for NYC Full',# Revision description.... use this, it's useful
     run_blocking = False)                # enable continue executing other commands before learning is finished